In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

torch_dtype = torch.bfloat16
checkpoint_dir = "model_gemma3-12b-full-finetuning"

attn_implementation = "flash_attention_2"
model = AutoModelForCausalLM.from_pretrained(
    checkpoint_dir,
    torch_dtype=torch_dtype,
    device_map="auto",
    attn_implementation=attn_implementation,
)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

print(f"Device: {model.device}")
print(f"DType: {model.dtype}")
print(f"model.config._attn_implementation: {model.config._attn_implementation}")

In [ ]:
question = "Who are you?"

# Format as a conversation
conversation = [{"role": "user", "content": question}]

prompt = pipe.tokenizer.apply_chat_template(
    conversation, tokenize=False, add_generation_prompt=True
)

outputs = pipe(prompt, max_new_tokens=256, disable_compile=True)
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")